In [1]:
require 'nn';
require 'cudnn';
require 'cunn';
require 'struct'
require 'image';

In [2]:
function traverse(writerParam, writerAct, layer, input)
    local layerName = torch.typename(layer)
    local output
    if layer.modules then
        if layerName == 'nn.Sequential' then
            output = input
            for i=1, #layer.modules do
                output = traverse(writerParam, writerAct, layer.modules[i], output)
            end
        elseif layerName == 'nn.ConcatTable' then
            output = {}
            for i=1, #layer.modules do
                table.insert(output, traverse(writerParam, writerAct, layer.modules[i], input))
            end
        else
            assert('Unknown container ' .. layerName)
        end
    else
        output = layer:forward(input)
        if layerName ~= 'nn.Identity' and layerName ~= 'nn.View' then
            params = {}
            counter = 0
            if layer.running_mean and layer.running_var then
                table.insert(params, layer.running_mean)
                table.insert(params, layer.running_var)
            end
            if layer.weight and layer.bias then
                table.insert(params, layer.weight)
                if torch.sum(layer.bias) ~= 0 then
                    table.insert(params, layer.bias)
                else
                    print("detect zero")
                end
            end
            for i=1, #params do
                param = params[i]
                counter = counter + param:nElement()
                for i=1, param:nElement() do
                    writerParam:write(struct.pack('<f', param:view(-1)[i]))
                end
            end
            if counter > 0 then
                print(("%s params: write %d bytes"):format(layerName, counter))
            end
            
            if #output:size() == 4 then
                writerAct:write(("%s %d %d %d\n"):format(layerName, output:size()[2], output:size()[3], output:size()[4]))
            else
                writerAct:write(("%s\n"):format(layerName))
            end
            for i=1,output:nElement() do
                writerAct:write(output:view(-1)[i])
                writerAct:write("\t")
            end
            writerAct:write("\n")
            print(("%s act: write %d bytes"):format(layerName, output:nElement()))
        end
    end
    return output
end 

In [3]:
data = torch.load('input.t7'):float()
modelCPU = cudnn.convert(torch.load('/home/minint/modelzoo/resnet152/model.t7'), nn):float()
modelCPU:evaluate()
torch.manualSeed(11)

In [4]:
image = data[1]:view(1, 3, 224, 224)
writerParam = assert(io.open('sixiao/param.bin', 'wb'))
writerAct = assert(io.open('sixiao/act.bin', 'wb'))
traverse(writerParam, writerAct, modelCPU, image)
writerParam:close()
writerAct:close()

detect zero	


nn.SpatialConvolution params: write 9408 bytes	


nn.SpatialConvolution act: write 802816 bytes	


nn.SpatialBatchNormalization params: write 256 bytes	


nn.SpatialBatchNormalization act: write 802816 bytes	


...

nn.CAddTable act: write 100352 bytes	


nn.ReLU act: write 100352 bytes	


nn.SpatialAveragePooling act: write 2048 bytes	


nn.Linear params: write 2049000 bytes	


nn.Linear act: write 1000 bytes	


In [8]:
modelCPU:get(#modelCPU).bias

0.01 *
-0.2244
-1.0511
-1.0846
-1.5553
 0.8360
 0.0734
-1.0321
 0.4000
 0.9090
-0.5566
-1.2695
-0.8785
-2.2761
-1.4642
-1.5511
-0.9296
-0.7791
-1.4805
 1.0375
-1.2024
 0.9199
 2.4019
-0.5844
 0.9269
-0.0720
 0.6964
 1.1858
-0.0073
 0.5451
-1.5340
-0.7740
-0.3871
-1.1786
-0.9663
 0.4412
-1.9984
-0.5670
-0.9770
 0.4616
-0.2214
 0.5074
-0.6768
 1.2103
-1.5497
 0.3324
 0.2964
 2.1876
-1.6819
-1.3816
-0.6093
 1.3474
-1.4438
 0.5744
 1.3054
 0.8231
 0.5648
 2.1051
-0.3394
 1.9205
-0.0832
 1.6256
 0.3924
-1.1956
 0.1125
 0.7319
 0.7176
-1.3110
-0.7244
 1.2687
-1.2604
-0.8171
 1.0471
-0.1581
 0.0217
-0.3873
 0.5148
-0.4981
 1.5628
 1.0027
 1.2349
-1.7508
 1.9105
-0.1848
-2.8862
-1.5502
 0.0524
-1.3319
-0.0419
 0.2656
 0.9331
-0.7317
-1.3097
 1.0686
-0.6694
-1.4265
-1.7266
-0.6721
-0.6128
-1.2575
-0.3244
-0.5547
-1.7996
-0.2475
 0.0949
 1.4759
-1.6062
-0.2139
-1.4594
-1.7985
-2.3627
-1.6771
 1.5061
 0.0470
-1.3497
-0.1874
-2.4485
-1.6256
-0.5745
-0.0536
 0.6912
-1.4606
-1.6614
-0.8970
-1.6034
 

1.7982
 0.3283
 0.5012
-1.1441
 1.6172
 0.1888
-0.5373
-0.0439
 0.2132
-1.8769
 0.5199
 1.9754
 1.8330
-0.5748
-1.6547
 1.3970
 1.2122
-0.1295
 1.1177
 0.7700
-0.4245
 1.1928
-1.0193
 0.1969
 0.2815
 0.0749
 1.1262
 0.1633
 1.4701
-2.6358
 0.9128
 1.1782
 0.7911
 0.5681
-0.0187
 0.3099
-1.2467
-0.4071
-1.1133
-0.7290
-0.0968
-0.0482
 0.3727
 0.0747
 0.1564
-2.0853
 1.0522
 0.3561
 1.0799
-0.3423
-0.4107
-0.0272
 1.3203
-1.6035
-1.0226
-0.6514
 1.7830
 0.1219
-0.6292
-0.7571
 1.0523
 0.6725
-0.3724
 0.0287
-0.0955
-0.2726
 0.0104
-0.2078
 0.3570
 0.5203
-0.8961
 2.4663
 0.7430
 0.4457
 0.1678
-2.1974
 1.7769
 0.5889
-0.6214
-0.3194
-0.5462
-0.5494
 1.3609
-0.3275
-0.8479
-0.1813
 1.0109
-1.2984
 0.5220
 1.1990
 0.9318
 0.1611
-0.3184
-0.5492
 0.7441
 0.2893
 0.2834
 1.2901
-0.6087
 0.5125
-2.0914
-0.7916
-0.4593
-0.3474
 0.2528
 0.9521
-0.2387
 0.9941
-0.9776
 1.0576
 0.6309
 1.6649
 2.8305
 1.3875
 0.6156
 1.9763
 1.4969
-1.0545
 1.1443
-0.1747
-1.6145
 1.5377
-0.0488
-0.5635
-0.4464
 